In [7]:
# Required Packages
try:
    from bs4 import BeautifulSoup
    import requests
    import csv
except ImportError:
    %%capture
    !pip install bs4
    !pip install requests
    from bs4 import BeautifulSoup
    import requests
    import csv

In [34]:
url="https://www.spotrac.com/mlb/payroll/2021"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
# print(soup.prettify()) # print the parsed data of html

### Find Table of interest on the webpage

In [35]:
mlb_table = soup.find("table", attrs={"class": "datatable rtable captracker"})

mlb_table_header = mlb_table.thead.find_all("tr") # Headers
mlb_table_data = mlb_table.tbody.find_all("tr")  # Rows

### Extract Information from the Table

In [36]:
# Get all the headings
headings = []
for th in mlb_table_header[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Rank', 'Team', 'Win%', 'Roster', '26-Man Payroll', 'Injured Reserve', 'Retained', 'Buried', 'Suspended', '2021 Total Payroll']


In [37]:
data = []
for tr in mlb_table_data: # find all tr's from table's tbody
    if tr.attrs:
        if tr.attrs["class"][0] == "average":
            continue
    row = {}
    # Each row is stored in the form of
    # row = {'Rank': '', 'Team': '',etc...}

    # find all td's in tr and zip it with headings
    for td, th in zip(tr.find_all("td"), headings):
        row[th] = td.text.replace('\n', '').strip()
        # Creating custom column for team code
        if th == 'Team':
            row['code'] = row[th][-3:]
            row[th] = row[th][:-3]
    data.append(row)
    
# Adding custom column to headings
headings.insert(2, 'code')
print(data)

[{'Rank': '1', 'Team': 'Los Angeles Dodgers', 'code': 'LAD', 'Win%': '0.654', 'Roster': '28', '26-Man Payroll': '$174,661,542', 'Injured Reserve': '$43,530,833', 'Retained': '$6,150,521', 'Buried': '$24,070,000', 'Suspended': '0-', '2021 Total Payroll': '$271,200,832'}, {'Rank': '2', 'Team': 'New York Yankees', 'code': 'NYY', 'Win%': '0.568', 'Roster': '30', '26-Man Payroll': '$141,518,753', 'Injured Reserve': '$48,217,362', 'Retained': '$12,129,712', 'Buried': '$13,040,875', 'Suspended': '0-', '2021 Total Payroll': '$205,669,863'}, {'Rank': '3', 'Team': 'New York Mets', 'code': 'NYM', 'Win%': '0.475', 'Roster': '28', '26-Man Payroll': '$154,565,754', 'Injured Reserve': '$33,933,570', 'Retained': '$9,524,946', 'Buried': '$12,006,000', 'Suspended': '0-', '2021 Total Payroll': '$201,189,189'}, {'Rank': '4', 'Team': 'Houston Astros', 'code': 'HOU', 'Win%': '0.586', 'Roster': '28', '26-Man Payroll': '$147,127,725', 'Injured Reserve': '$39,870,546', 'Retained': '$4,134,205', 'Buried': '$7,5

### Exporting Data to Excel

In [38]:
with open("mlb.csv", 'w', newline = '') as out_file:
    writer = csv.DictWriter(out_file, headings)
    writer.writeheader()
    writer.writerows(data)